## Parsing market.ya.ru for review of mobile phones

In [1]:
%load_ext autoreload
%autoreload 2
import requests
import bs4
import functions as f
from sklearn.externals import joblib
import re

In [2]:
data = joblib.load('DUMP_data.pkl')
print ('At this moment there are {} reviews in data'.format(len(data)))

At this moment there are 57315 reviews in data


In [29]:
# pattern to parse
#https://market.yandex.ru/catalog/54726/list?hid=91502&local-offers-first=0&viewtype=list&how=opinions&page=2
# get new products
productIds = []
for pageOfProducts in range (1,50):
    url = 'https://market.yandex.ru/catalog/54726/list?hid=91922&local-offers-first=0&viewtype=list&how=aprice&page={}'.format(pageOfProducts)
    req = requests.get(url)
    parser = bs4.BeautifulSoup(req.text, 'lxml')
    
    linksRaws = [a.get("href") for a in parser.findAll('a', attrs={'class':'n-snippet-card2__image link'})]
    if len(linksRaws) == 0:
        break
    for linkRaw in linksRaws:
        productIds.append((re.search('/product/(.*)\?show-', linkRaw)).group(1))
print ('Quantity of products: {}'.format(len(productIds)))


Quantity of products: 2112


In [30]:
#delete from productIds elems that are not in data
alreadyParsedProductsIdx = []
for idx in range (0,len(data)):
    alreadyParsedProductsIdx.append(data[idx][0])

productIds = list(set(productIds)-set(alreadyParsedProductsIdx))
print ('Calculated and will be parsed {} products'.format(len(productIds)))

lastSuccessedParsedProductIdx = 0

Calculated and will be parsed 399 products


In [31]:
parsingDetected = False
for productId in productIds[lastSuccessedParsedProductIdx:]:
    #it can be improved to save only new reviews for existing products
    countReviewsForProduct = 0
    for page in range (1,150):
        url = 'https://market.yandex.ru/product/{}/reviews?page={}'.format(productId, page)
        
        #make connection more stable 
        for _ in range(0,10):
            try:
                req = requests.get(url)
                parser = bs4.BeautifulSoup(req.text, 'lxml')
                break
            except:
                print ('Problem with network. Trying to reconnect...')
                pass
        
        #this block should be unparsed if you are sure that all your products HAVE reviews
        #check if ya detects my parsing
        #if len(parser.findAll('div', attrs={'class':'n-product-review-item'})) == 0 and page == 1:
        #    parsingDetected = True
        #    print ('Probably, Ya detected my parsing. Parsing has stoped')
        #    break
        #check if there are any reviews
        if len(parser.findAll('div', attrs={'class':'n-product-review-item'})) == 0:
            break
    
        for fullReview in parser.findAll('div', attrs={'class':'n-product-review-item'}):
            countReviewsForProduct += 1
            data.append(f.get_review_details(fullReview, productId))
    if parsingDetected:
        # save for resume from this elemen next time
        #lastSuccessedParsedProductIdx = productIds.index(productId)
        print ('Last successed parsed product position was = {}. Product = {}'.format(lastSuccessedParsedProductIdx, productId))
        parsingDetected = False
        break
    
    ### realTime saving!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    if countReviewsForProduct>0: #if we got something new to save
        print ('Count down: {}.Total: {}. Product: {}. Parsed and saved to data. Reviews pages = {}, count reviews = {}.'.
           format(len(productIds) - productIds.index(productId), len(data), productId, page-1, countReviewsForProduct))
        _ = joblib.dump(data, 'DUMP_data.pkl', compress=9)
    ### realTime saving!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    
    if countReviewsForProduct==0 and lastSuccessedParsedProductIdx%20==0:
        print ('Count down: {}'.format(len(productIds) - productIds.index(productId)))
    
    lastSuccessedParsedProductIdx += 1
    #print ('Now working on parsing product on total position {}'.format(productIds.index(productId)))
print ('Done')    

## Manual data save if needed

In [25]:
#save/load data
#_ = joblib.dump(list(set(data)), 'DUMP_data.pkl', compress=9)
